# Data Preprocessing (DWMW17)

In [ ]:
from google.colab import drive
drive.mount('/drive')
path = '/drive/My Drive/CSCI544 Project/'

Mounted at /drive


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter, OrderedDict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, MultiStepLR, CyclicLR
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

In [ ]:
# if torch.cuda.is_available():  
#   dev = "cuda:0"
#   torch.cuda.set_device(0)
# else:  
#   dev = "cpu"
device = torch.device('cpu')

In [ ]:
# Used DWMW17 from https://github.com/t-davidson/hate-speech-and-offensive-language/tree/master/data
data_path = 'https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv'
data = pd.read_csv(data_path)
df = data[['class', 'tweet']].copy()

In [ ]:
len(df)

24783

In [ ]:
# preprocessing
df['tweet'] = df['tweet'].apply(lambda x:x.lower())
punctuation_signs = list("?:!.,;")
df['tweet'] = df['tweet']

for punct_sign in punctuation_signs:   
    df['tweet'] = df['tweet'].str.replace(punct_sign, '')

df['tweet'] = df['tweet'].apply(lambda x: x.replace('\n', ' '))
df['tweet'] = df['tweet'].apply(lambda x: x.replace('\t', ' '))
df['tweet'] = df['tweet'].str.replace("    ", " ")
df['tweet'] = df['tweet'].str.replace('"', '')
df['tweet'] = df['tweet'].str.replace("'s", "")

<ipython-input-27-29da0877ee92>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['tweet'] = df['tweet'].str.replace(punct_sign, '')


In [ ]:
# remove stop words
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    df['tweet'] = df['tweet'].str.replace(regex_stopword, '')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-28-5e5fe2b64a04>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df['tweet'].str.replace(regex_stopword, '')


In [ ]:
df['tweet'] = df['tweet'].str.split()

In [ ]:
glove_model = {}
with open(path + 'glove.6B.100d','r') as f:
    for line in f:
        line = line.split()
        word = line[0]
        embedding = np.array(line[1:], dtype=np.float64)
        glove_model[word] = embedding

In [ ]:
# # make encoding in X match glove embedding
vecs = np.zeros((len(glove_model), 100), dtype=np.float64)
i = 0
for word, embedding in glove_model.items():
    vecs[i] = embedding
    i += 1

pad_vec = np.zeros((1,100))   # vector for padding

unk_vec = np.mean(vecs, axis=0) # <unk>

glove_embeddings = np.vstack((pad_vec, unk_vec, vecs))
glove_vocab = list(glove_model.keys())
glove_vocab.insert(0, '<unk>')
glove_vocab.insert(0, '<pad>')
word_index = {glove_vocab[i]:i for i in range(len(glove_vocab))}

In [ ]:
X = [0] * len(df)

for index, row in df.iterrows():
  X[index] = []
  for word in row['tweet']:
    if word in glove_vocab:
      X[index].append(word_index[word])
    else:
      X[index].append(word_index['<unk>'])

In [ ]:
# count = 0
# for index, row in df.iterrows():
#   X[count] = []
#   for word in row['tweet']:
#     if word in glove_vocab:
#       X[count].append(word_index[word])
#     else:
#       X[count].append(word_index['<unk>'])
#   count += 1

In [ ]:
y = df['class']

In [ ]:
indices = np.arange(len(df))
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size = 0.2, stratify=y, random_state=17)
X_val, X_test, y_val, y_test, indices_val, indices_test = train_test_split(X_test, y_test, indices_test, test_size = 0.5, stratify=y_test, random_state=17)

In [ ]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

class TrainData(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = torch.transpose(pad_sequence([torch.Tensor(i) for i in X], padding_value=0), 0, 1).to(device)
        self.y = y
        self.lengths = [len(i) for i in X]
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        tweet = self.X[index]
        label = self.y.iloc[index]
        length = self.lengths[index]
            
        return tweet, length, label


class TestData(Dataset):
    def __init__(self, X, transform=None):
      self.X = torch.transpose(pad_sequence([torch.Tensor(i) for i in X], padding_value=0), 0, 1).to(device)
      self.lengths = [len(i) for i in X]

      self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        tweet = self.X[index]
        length = self.lengths[index]
            
        return tweet, length

In [ ]:
train_data = TrainData(X_train, y_train, transform=transforms.ToTensor())
val_data = TrainData(X_val, y_val, transform=transforms.ToTensor())
batch_size = 20
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Model (DWMW17)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM(nn.Module):
    def __init__(self, input_size, embed_dim, hidden_size, num_layers, num_classes, dropout):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(glove_embeddings).float())
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.linear1 = nn.Linear(hidden_size, 128)
        self.elu = nn.ELU()
        self.fc = nn.Linear(128 , num_classes)
        
    def forward(self, x, lengths):
        # Initialize hidden states and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # one forward, one backward, so *2
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x = x.int().to(device)
        out = self.embedding(x)
        out = pack_padded_sequence(out, lengths, batch_first=True, enforce_sorted=False)
        out, _ = self.lstm(out, (h0, c0))
        out, _ = pad_packed_sequence(out, batch_first=True)
        out = self.linear1(out)
        out = self.dropout(out)
        out = self.elu(out)
        out = self.fc(out)
        out = out[:, -1, :]

        return out

In [ ]:
n_hidden = 256
n_input = len(glove_embeddings)
n_embed_dim = 100
n_layers = 1
n_classes = 3 # output layer
dropout = 0.33
lstm = LSTM(n_input, n_embed_dim, n_hidden, n_layers, n_classes, dropout)
lstm.to(device)

In [ ]:
learning_rate = 0.5
n_epochs = 30
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)
scheduler = MultiStepLR(optimizer, milestones=[10, 20, 50], gamma=0.5)
# scheduler = CyclicLR(optimizer, base_lr=0.005, max_lr=0.5)
train_loss_min = np.Inf # initialize minimum validation loss

for epoch in range(n_epochs):
  # initialize train and validation loss
  train_loss = 0
  valid_loss = 0


  # train
  # mini-batch gradient descent
  lstm.train()
  for i, (data, lengths, target) in enumerate(train_loader):
    # forward
    output = lstm(data, lengths)
    loss = criterion(output, target)
    optimizer.zero_grad()
    # backward
    loss.backward()
    # update parameters
    optimizer.step()
    # track training loss
    train_loss += loss.item()
    # scheduler.step()


  # evaluation
  lstm.eval()
  for i, (data,lengths, target) in enumerate(valid_loader):
      # forward pass
      output = lstm(data, lengths)
      # calculate the loss
      loss = criterion(output, target)
      # track validation loss
      valid_loss += loss.item()


  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(valid_loader.dataset)
  
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}\tLearning Rate: {:.3f}'.format(
      epoch+1, 
      train_loss,
      valid_loss,
      scheduler.get_last_lr()[0]
      ))
  
  if train_loss <= train_loss_min:
      print('Training loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      train_loss_min,
      train_loss))
      torch.save(lstm.state_dict(), 'lstm_model.pt')
      train_loss_min = train_loss

  scheduler.step()

In [ ]:
torch.save(lstm.state_dict(), path+'lstm_model(dwmw17).pt')

In [ ]:
# lstm.load_state_dict(torch.load(path+'lstm_model.pt'))
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def test():
  test_data = TestData(X_test, transform=transforms.ToTensor())
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, 
      num_workers=num_workers)
  y_pred = []
  lstm.eval()
  with torch.no_grad():
      for data, lengths in test_loader:
          outputs = lstm(data, lengths)
          _, predicted = torch.max(outputs.data, 1)
          y_pred.append(predicted.tolist()[0])

  print("Test Accuracy: " + str(accuracy_score(list(y_test), y_pred)))
  print("Test F1 Score: "+ str(f1_score(list(y_test), y_pred, average='micro')))

In [ ]:
test()

In [ ]:
def predict():
  test_data = TestData(X, transform=transforms.ToTensor())
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, 
      num_workers=num_workers)
  y_pred = []
  lstm.eval()
  with torch.no_grad():
      for data, lengths in test_loader:
          outputs = lstm(data, lengths)
          _, predicted = torch.max(outputs.data, 1)
          y_pred.append(predicted.tolist()[0])
  return y_pred

In [ ]:
predict()

# Data Preprocessing (FDCL18)

In [ ]:
fdcl18_data = pd.read_csv(path+'FDCL18.csv', delimiter='\t', nrows=None, skiprows=2, header=None, names=['tweet', 'label', 'count'])

In [ ]:
df1 = fdcl18_data[['tweet', 'label']].copy()
df1.loc[df1.label == 'hateful', 'label'] = 0
df1.loc[df1.label == 'abusive', 'label'] = 1
df1.loc[df1.label == 'spam', 'label'] = 2
df1.loc[df1.label == 'normal', 'label'] = 3

In [ ]:
# preprocessing
df1['tweet'] = df1['tweet'].apply(lambda x:x.lower())
punctuation_signs = list("?:!.,;")
df1['tweet'] = df1['tweet']

for punct_sign in punctuation_signs:   
    df1['tweet'] = df1['tweet'].str.replace(punct_sign, '')

df1['tweet'] = df1['tweet'].apply(lambda x: x.replace('\n', ' '))
df1['tweet'] = df1['tweet'].apply(lambda x: x.replace('\t', ' '))
df1['tweet'] = df1['tweet'].str.replace("    ", " ")
df1['tweet'] = df1['tweet'].str.replace('"', '')
df1['tweet'] = df1['tweet'].str.replace("'s", "")

# remove stop words
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    df1['tweet'] = df1['tweet'].str.replace(regex_stopword, '')

<ipython-input-12-02dd1e57637d>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df1['tweet'] = df1['tweet'].str.replace(punct_sign, '')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-12-02dd1e57637d>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['tweet'] = df1['tweet'].str.replace(regex_stopword, '')


In [ ]:
df1['tweet'] = df1['tweet'].str.split()

In [ ]:
X = [0] * len(df1)

for index, row in df1.iterrows():
  X[index] = []
  for word in row['tweet']:
    if word in glove_vocab:
      X[index].append(word_index[word])
    else:
      X[index].append(word_index['<unk>'])

In [ ]:
y = df1['label']

In [ ]:
indices = np.arange(len(X))
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, indices, test_size = 0.2, stratify=y, random_state=17)
X_val, X_test, y_val, y_test, indices_val, indices_test = train_test_split(X_test, y_test, indices_test, test_size = 0.5, stratify=y_test, random_state=17)

In [ ]:
train_data = TrainData(X_train, y_train, transform=transforms.ToTensor())
val_data = TrainData(X_val, y_val, transform=transforms.ToTensor())
batch_size = 60
num_workers = 0
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Model (FDCL18)

In [ ]:
n_hidden = 256
n_input = len(glove_embeddings)
n_embed_dim = 100
n_layers = 1
n_classes = 4 # output layer
dropout = 0.33
lstm = LSTM(n_input, n_embed_dim, n_hidden, n_layers, n_classes, dropout)
lstm.to(device)

LSTM(
  (embedding): Embedding(400002, 100)
  (lstm): LSTM(100, 256, batch_first=True)
  (dropout): Dropout(p=0.33, inplace=False)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (fc): Linear(in_features=128, out_features=4, bias=True)
)

In [ ]:
learning_rate = 0.5
n_epochs = 20
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)
scheduler = MultiStepLR(optimizer, milestones=[10, 20, 50], gamma=0.5)
# scheduler = CyclicLR(optimizer, base_lr=0.005, max_lr=0.5)
train_loss_min = np.Inf # initialize minimum validation loss

for epoch in range(n_epochs):
  # initialize train and validation loss
  train_loss = 0
  valid_loss = 0


  # train
  # mini-batch gradient descent
  lstm.train()
  for i, (data, lengths, target) in enumerate(train_loader):
    # forward
    output = lstm(data, lengths)
    loss = criterion(output, target)
    optimizer.zero_grad()
    # backward
    loss.backward()
    # update parameters
    optimizer.step()
    # track training loss
    train_loss += loss.item()
    # scheduler.step()


  # evaluation
  lstm.eval()
  for i, (data,lengths, target) in enumerate(valid_loader):
      # forward pass
      output = lstm(data, lengths)
      # calculate the loss
      loss = criterion(output, target)
      # track validation loss
      valid_loss += loss.item()


  train_loss = train_loss/len(train_loader.dataset)
  valid_loss = valid_loss/len(valid_loader.dataset)
  
  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}\tLearning Rate: {:.3f}'.format(
      epoch+1, 
      train_loss,
      valid_loss,
      scheduler.get_last_lr()[0]
      ))
  
  if train_loss <= train_loss_min:
      print('Training loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
      train_loss_min,
      train_loss))
      torch.save(lstm.state_dict(), 'lstm_model.pt')
      train_loss_min = train_loss

  scheduler.step()

Epoch: 1 	Training Loss: 0.018595 	Validation Loss: 0.018690	Learning Rate: 0.500
Training loss decreased (inf --> 0.018595).  Saving model ...
Epoch: 2 	Training Loss: 0.018527 	Validation Loss: 0.018560	Learning Rate: 0.500
Training loss decreased (0.018595 --> 0.018527).  Saving model ...
Epoch: 3 	Training Loss: 0.018458 	Validation Loss: 0.018463	Learning Rate: 0.500
Training loss decreased (0.018527 --> 0.018458).  Saving model ...
Epoch: 4 	Training Loss: 0.018444 	Validation Loss: 0.018467	Learning Rate: 0.500
Training loss decreased (0.018458 --> 0.018444).  Saving model ...
Epoch: 5 	Training Loss: 0.018424 	Validation Loss: 0.018536	Learning Rate: 0.500
Training loss decreased (0.018444 --> 0.018424).  Saving model ...
Epoch: 6 	Training Loss: 0.018426 	Validation Loss: 0.018495	Learning Rate: 0.500
Epoch: 7 	Training Loss: 0.018410 	Validation Loss: 0.018455	Learning Rate: 0.500
Training loss decreased (0.018424 --> 0.018410).  Saving model ...
Epoch: 8 	Training Loss: 0.01

In [ ]:
torch.save(lstm.state_dict(), path+'lstm_model(fdcl18).pt')

In [ ]:
# lstm.load_state_dict(torch.load(path+'lstm_model(fdcl18).pt'))
test()
y_pred = predict()

Test Accuracy: 0.7652
Test F1 Score: 0.7652


In [ ]:
df1['pred'] = y_pred
with open(path+'lstm_pred(fdcl18).csv', 'w') as f:
  f.write('tweet,label,pred')
  for index, row in df1.iterrows():
    f.write(str(row['tweet'])+ ',' + str(row['label']) + ','+ str(row['pred']))

# LSTM Bias Evalutaion (DWMW16)

In [ ]:
# African-American, Hispanic, Asian, and White topics,
aae = np.genfromtxt(path+'aae.txt', delimiter=',')
df['race'] = aae.argmax(axis=1)

In [ ]:
lstm_result = df.copy()
lstm_result = lstm_result.reindex(np.arange(len(X)))
lstm_result.loc[indices_train, 'data_type'] = 'train'
lstm_result.loc[indices_val, 'data_type'] = 'val'
lstm_result.loc[indices_test, 'data_type'] = 'test'
lstm_result['pred'] = y_pred
lstm_result = lstm_result.loc[lstm_result['data_type']=='test'][['class', 'pred', 'race']]
aae_group = lstm_result.loc[lstm_result['race'] == 0]
other_group = lstm_result.loc[lstm_result['race'] != 0]

In [ ]:
def fpr(cm):
  FP = cm.sum(axis=0) - np.diag(cm)  
  FN = cm.sum(axis=1) - np.diag(cm)
  TP = np.diag(cm)
  TN = cm.sum() - (FP + FN + TP)
  FPR = FP/(FP+TN)
  return FPR

In [ ]:
from sklearn.metrics import confusion_matrix
print("LSTM Bias Evaluation (DWMW16): ")
print('\tHate Speech' + ' Offensive' + '  Neither')
aae_cm = confusion_matrix(aae_group['class'], aae_group['pred'])
print("AAE" + '\t' + str(fpr(aae_cm)))
other_cm = confusion_matrix(other_group['class'], other_group['pred'])
print("Non-AAE" + '\t' + str(fpr(other_cm)))

LSTM Bias Evaluation: 
	Hate Speech Offensive  Neither
AAE	[0.00077519 0.65322581 0.02340094]
Non-AAE	[0.01912046 0.34862385 0.08076923]


# Bert Bias Evaluation (DWMW16)

In [ ]:
bert = pd.read_csv(path+'preds.csv')
bert['race'] = aae.argmax(axis=1)
bert = bert.loc[bert['data_type']=='test'][['label', 'pred', 'race']]

In [ ]:
aae_group = bert.loc[bert['race'] == 0]
other_group = bert.loc[bert['race'] != 0]

In [ ]:
from sklearn.metrics import confusion_matrix
print("BERT Bias Evaluation: ")
print('\tHate Speech' + ' Offensive' + '  Neither')
aae_cm = confusion_matrix(aae_group['label'], aae_group['pred'])
print("AAE" + '\t' + str(fpr(aae_cm)))
other_cm = confusion_matrix(other_group['label'], other_group['pred'])
print("Non-AAE" + '\t' + str(fpr(other_cm)))

# LSTM Bias Evaluation (FDCL18)

In [ ]:
aae = np.genfromtxt(path+'fdcl18_aae.csv', delimiter=',')
df1['race'] = aae.argmax(axis=1)

In [ ]:
lstm_result = df1.copy()
lstm_result.loc[indices_train, 'data_type'] = 'train'
lstm_result.loc[indices_val, 'data_type'] = 'val'
lstm_result.loc[indices_test, 'data_type'] = 'test'
lstm_result['pred'] = y_pred
# lstm_result['race'] = aae.argmax(axis=1)
lstm_result = lstm_result.loc[lstm_result['data_type']=='test'][['label', 'pred', 'race']]
aae_group = lstm_result.loc[lstm_result['race'] == 0]
other_group = lstm_result.loc[lstm_result['race'] != 0]

In [ ]:
from sklearn.metrics import confusion_matrix
print("LSTM Bias Evaluation (FDCL18): ")
print('\tHateful' + ' Abusive' + '  Spam' + '  Normal')
aae_cm = confusion_matrix(list(aae_group['label']), list(aae_group['pred']))
print("AAE" + '\t' + str(fpr(aae_cm)))
other_cm = confusion_matrix(list(other_group['label']), list(other_group['pred']))
print("Non-AAE" + '\t' + str(fpr(other_cm)))

LSTM Bias Evaluation (FDCL18): 
	Hateful Abusive  Spam  Normal
AAE	[0.         0.38049713 0.08803828 0.05355191]
Non-AAE	[0.         0.08651287 0.14631886 0.17216216]


# Bert Bias Evaluation (FDCL18)

In [ ]:
bert = pd.read_csv(path+'FDCL_results(bert).csv')
bert['race'] = aae.argmax(axis=1)
bert = bert.loc[bert['data_type']=='test'][['label', 'pred', 'race']]

In [ ]:
aae_group = bert.loc[bert['race'] == 0]
other_group = bert.loc[bert['race'] != 0]

In [ ]:
from sklearn.metrics import confusion_matrix
print("BERT Bias Evaluation: ")
print('\tHateful' + ' Abusive' + '  Spam' + '  Normal')
aae_cm = confusion_matrix(aae_group['label'], aae_group['pred'])
print("AAE" + '\t' + str(fpr(aae_cm)))
other_cm = confusion_matrix(other_group['label'], other_group['pred'])
print("Non-AAE" + '\t' + str(fpr(other_cm)))

BERT Bias Evaluation: 
	Hateful Abusive  Spam  Normal
AAE	[0.0464666  0.19311663 0.0507177  0.04918033]
Non-AAE	[0.01676505 0.04658385 0.06355932 0.19972973]


# References:
https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal